# 第3章: 正規表現
---
Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz)がある．
- 1行に1記事の情報がJSON形式で格納される
- 各行には記事名が"title"キーに，記事本文が"text"キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
- ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [1]:
!wget 'http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz'

--2017-02-01 18:14:07--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz
Resolving www.cl.ecei.tohoku.ac.jp... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3605822 (3.4M) [application/x-gzip]
Saving to: 'jawiki-country.json.gz'

jawiki-country.json 100%[===================>]   3.44M  5.95MB/s    in 0.6s    

2017-02-01 18:14:08 (5.95 MB/s) - 'jawiki-country.json.gz' saved [3605822/3605822]



In [2]:
%%file q20.py
import json
import sys


for line in sys.stdin:
    wiki_dict = json.loads(line)
    if wiki_dict['title'] == 'イギリス':
        print(wiki_dict.get('text'))


Overwriting q20.py


In [4]:
!gzcat jawiki-country.json.gz | python q20.py > uk.txt

In [5]:
!head uk.txt

{{redirect|UK}}
{{基礎情報 国
|略名 = イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>


## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [26]:
%%file q21.py
import sys


for line in sys.stdin:
    lowered = line.lower()
    if lowered.startswith('[[category'):
        print(line.rstrip())


Overwriting q21.py


In [10]:
!python q21.py < uk.txt

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [25]:
%%file q22.py
import sys


for line in sys.stdin:
    print(line.lower().lstrip("[[category:").rstrip("]]\n"))


Overwriting q22.py


In [24]:
!python q21.py < uk.txt | python q22.py

イギリス|*
英連邦王国|*
8加盟国
欧州連合加盟国
海洋国家
君主国
島国|くれいとふりてん
1801年に設立された州・地域


## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [40]:
%%file q23.py
# ==国名== → 国名 レベル1 
import sys

for line in sys.stdin:
    if line.startswith('=='):
        sec_name = line.strip('= \n')
        level = int(line.count('=')/2 - 1)
        print(sec_name, 'レベル'+str(level))


Overwriting q23.py


In [39]:
!python q23.py < uk.txt | head

国名 レベル1
歴史 レベル1
地理 レベル1
気候 レベル2
政治 レベル1
外交と軍事 レベル1
地方行政区分 レベル1
主要都市 レベル2
科学技術 レベル1
経済 レベル1


## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [4]:
%%file q24.py
# [[File:Battle of Waterloo 1815.PNG| のようになっている
import re
import sys

pat = re.compile(r'([fF]ile:|ファイル:)(?P<filename>.+?)\|')
for line in sys.stdin:
    for m in pat.finditer(line):
        print(m.group('filename'))


Overwriting q24.py


In [2]:
!python q24.py < uk.txt | head

Royal Coat of Arms of the United Kingdom.svg
Battle of Waterloo 1815.PNG
The British Empire.png
Uk topo en.jpg
BenNevis2005.jpg
Elizabeth II greets NASA GSFC employees, May 8, 2007 edit.jpg
Palace of Westminster, London - Feb 2007.jpg
David Cameron and Barack Obama at the G20 Summit in Toronto.jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
Scotland Parliament Holyrood.jpg


## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [5]:
%%file q25.py
"""
{{基礎情報 国
|略名 = イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
"""
import sys
import json


def main():
    dic = extract_baseinf(sys.stdin)
    sys.stdout.write(json.dumps(dic, ensure_ascii=False))
    for k, v in dic.items():
        print(k, v, file=sys.stderr)


def extract_baseinf(fi):
    baseinf = {}
    isbaseinf = False
    for line in fi:
        if isbaseinf:
            if line.startswith('}}'):
                return baseinf

            elif line[0] == '|':
                templis = line.strip('|\n').split('=')
                key = templis[0].strip()
                value = "=".join(templis[1:])
                baseinf[key] = value
                
            else:
                baseinf[key] += '\n{}'.format(line.rstrip('\n'))

        elif line.startswith('{{基礎情報'):
            isbaseinf = True


if __name__ == '__main__':
    main()


Overwriting q25.py


In [18]:
!python q25.py < uk.txt > uk_baseinf.dict.json

日本語国名  グレートブリテン及び北アイルランド連合王国
確立年月日4  [[1927年]]
GDP順位MER  5
GDP値元  1兆5478億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom]</ref>
通貨  [[スターリング・ポンド|UKポンド]] (&pound;)
GDP統計年元  2012
面積順位  76
ISO 3166-1  GB / GBR
元首等肩書  [[イギリスの君主|女王]]
略名  イギリス
GDP値MER  2兆4337億<ref name="imf-statistics-gdp" />
注記  <references />
人口統計年  2011
国章画像  [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
確立年月日2  [[1707年]]
GDP/人  36,727<ref name="imf-statistics-gdp" />
確立形態1  [[イングランド王国]]／[[スコットランド王国]]<br />（両国とも[[連合法 (1707年)|1707年連合法]]まで）
面積値  244,820
確立年月日1  [[927年]]／[[843年]]
建国形態  建国
GDP値  2兆3162億<ref name="imf-statistics-gdp" />
国章リンク  （[[イギリスの国章|国章]]）
水面積率  1.3%
人口順位  22
通貨コード  GBP
公式国名  {{lang|en|United Kingdom of Great Britain and Northern Ireland

## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [41]:
%%file q26.py
"""
'が2, 3, 5個連続していた場合、'を取り除く
"""
import json
import re
import sys
from pprint import pprint

def main():
    dic = json.loads(sys.stdin.read())
    dic = no_emphasis(dic)
    pprint(dic)
    # sys.stdout.write(json.dumps(dic))

def no_emphasis(dic):
    for key, value in dic.items():
        for n in (5, 3, 2):
            eliminated = value.split("'" * n)
            div, mod = divmod(len(eliminated), 2)
            if div > 0 and mod != 0:
                value = ''.join(eliminated)
                dic[key] = value
                break
        # print(key, value,file=sys.stderr)
    return dic

if __name__ == '__main__':
    main()


Overwriting q26.py


In [42]:
!python q26.py < uk_baseinf.dict.json

{'GDP/人': ' 36,727<ref name="imf-statistics-gdp" />',
 'GDP値': ' 2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': ' 2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': ' 1兆5478億<ref '
          'name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= '
          'IMF>Data and Statistics>World Economic Outlook Databases>By '
          'Countrise>United Kingdom]</ref>',
 'GDP統計年': ' 2012',
 'GDP統計年MER': ' 2012',
 'GDP統計年元': ' 2012',
 'GDP順位': ' 6',
 'GDP順位MER': ' 5',
 'ISO 3166-1': ' GB / GBR',
 'ccTLD': ' [[.uk]] / [[.gb]]<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': ' 63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm '
        'United Nations Department of Economic and Social Affairs>Population '
        'Division>Data>Population>Total Population]</ref>',
 '人口大きさ': ' 1 E7',
 '人口密度値': ' 246',
 '人口統計年': ' 2011',
 '人口順位

## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: [マークアップ早見表](https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [69]:
%%file q27.py
"""
[[記事名]]
[[記事名|表示文字]]
[[記事名#節名|表示文字]] 
"""
import json
import re
import sys
from pprint import pprint


from q26 import no_emphasis


def main():
    dic = json.loads(sys.stdin.read())
    dic = no_emphasis(dic)
    dic = eliminate_link(dic)
    pprint(dic)
    # sys.stdout.write(json.dumps(dic))


def eliminate_link(dic):
    pat = re.compile(r"""
        \[\[        # [[
        ([^|]+\|)*  # 記事名|　この部分は無かったり繰り返されたりする
        ([^]]+)\]\] # 表示文字 patにマッチした部分をこいつに置換する
    """, re.VERBOSE)
    for key, value in dic.items():
        value = pat.sub(r'\2', value)
        dic[key] = value
    return dic

if __name__ == '__main__':
    main()


Overwriting q27.py


In [70]:
!python q27.py < uk_baseinf.dict.json

{'GDP/人': ' 36,727<ref name="imf-statistics-gdp" />',
 'GDP値': ' 2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': ' 2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': ' 1兆5478億<ref '
          'name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= '
          'IMF>Data and Statistics>World Economic Outlook Databases>By '
          'Countrise>United Kingdom]</ref>',
 'GDP統計年': ' 2012',
 'GDP統計年MER': ' 2012',
 'GDP統計年元': ' 2012',
 'GDP順位': ' 6',
 'GDP順位MER': ' 5',
 'ISO 3166-1': ' GB / GBR',
 'ccTLD': ' .uk / .gb<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': ' 63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm '
        'United Nations Department of Economic and Social Affairs>Population '
        'Division>Data>Population>Total Population]</ref>',
 '人口大きさ': ' 1 E7',
 '人口密度値': ' 246',
 '人口統計年': ' 2011',
 '人口順位': ' 22'

## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [37]:
%%file q28.py
import json
import sys
from pprint import pprint

from pypandoc import convert


def main():
    dic = json.loads(sys.stdin.read())
    for key, value in dic.items():
        dic[key] = convert(value, 'plain', format='mediawiki').rstrip()
    
    pprint(dic)

if __name__ == '__main__':
    main()

Overwriting q28.py


In [38]:
!python q28.py < uk_baseinf.dict.json

{'GDP/人': '36,727[1]\n\n[1]',
 'GDP値': '2兆3162億[1]\n\n[1]',
 'GDP値MER': '2兆4337億[1]\n\n[1]',
 'GDP値元': '1兆5478億[1]\n'
          '\n'
          '[1] '
          '[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= '
          'IMF>Data and Statistics>World Economic Outlook Databases>By '
          'Countrise>United Kingdom]',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO 3166-1': 'GB\xa0/\xa0GBR',
 'ccTLD': '.uk\xa0/\xa0.gb[1]\n\n[1] 使用は.ukに比べ圧倒的少数。',
 '人口値': '63,181,775[1]\n'
        '\n'
        '[1] United Nations Department of Economic and Social '
        'Affairs>Population Division>Data>Population>Total Population',
 '人口大きさ': '1\xa0E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': 'エリザベス2世',
 '元首等肩書': '女王',
 '公式国名': '[1]\n'
         '\n'
  

## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: [MediaWiki](https://www.mediawiki.org/wiki/API:Main_page/ja) APIの[imageinfo](https://www.mediawiki.org/wiki/API:Properties/ja#imageinfo_.2F_ii)を呼び出して，ファイル参照をURLに変換すればよい）

In [86]:
%%file q29.py
import sys
import json


import requests

baseinf = json.loads(sys.stdin.read())
r = requests.get('https://commons.wikimedia.org/w/api.php',
                 {'action': 'query', 'prop': 'imageinfo', 'iiprop': 'url',
                  'format': 'json', 'titles': 'File:{}'.format(baseinf['国旗画像'])})
data = r.json()
print(data['query']['pages']['347935']['imageinfo'][0]['url'])

Overwriting q29.py


In [87]:
!python q29.py < uk_baseinf.dict.json

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
